In [1]:
import pandas as pd
import numpy as np
import pandas_profiling
import ydata_profiling
import math

In [2]:
crypto_punks_df = pd.read_csv('data/punks.csv')

In [3]:
crypto_punks_df.head(10)

,Unnamed: 0,Type,From,To,Amount,Txn,Punk
0,0,Claimed,NaN,0xc352b5,NaN,"Jun 23, 2017",0
1,1,Bid,0x948a10,NaN,0.67Ξ ($219),"Jun 23, 2017",0
2,2,Bid,0x00d7c9,NaN,0.98Ξ ($320),"Jun 23, 2017",0
3,3,Sold,0xc352b5,0x00d7c9,0.98Ξ ($320),"Jun 23, 2017",0
4,4,Bid,0x717403,NaN,0.05Ξ ($14),"Jun 25, 2017",0
5,5,Bid,0xa0a59c,NaN,1.30Ξ ($369),"Jul 02, 2017",0
6,6,Bid,0x7760e0,NaN,1.30Ξ ($369),"Jul 02, 2017",0
7,7,Bid,0xa0a59c,NaN,1.52Ξ ($422),"Jul 03, 2017",0
8,8,Bid,0x7760e0,NaN,1.53Ξ ($422),"Jul 03, 2017",0
9,9,Bid,0xa0a59c,NaN,1.60Ξ ($386),"Jul 07, 2017",0


In [4]:
crypto_punks_df.Punk.value_counts()

1512    138
240     111
1605    108
406     101
339     101
       ... 
313       1
1250      1
1298      1
140       1
351       1
Name: Punk, Length: 1635, dtype: int64

In [5]:
crypto_punks_df.Type.value_counts(normalize=True)

Offered            0.339316
Bid                0.190581
Bid Withdrawn      0.116988
Transfer           0.114289
Sold               0.083942
Claimed            0.081742
Offer Withdrawn    0.051245
(Wrap)             0.011699
(Unwrap)           0.009999
Bid *              0.000200
Name: Type, dtype: float64

In [6]:
crypto_punks_df.Punk.value_counts(normalize=True)

1512    0.006899
240     0.005549
1605    0.005399
406     0.005049
339     0.005049
          ...   
313     0.000050
1250    0.000050
1298    0.000050
140     0.000050
351     0.000050
Name: Punk, Length: 1635, dtype: float64

In [7]:
crypto_punks_df.From.value_counts(normalize=True)

0xc352b5     0.060066
Pranksy      0.034581
0x717403     0.027475
WrappedC…    0.018948
0xcbd482     0.014590
               ...   
markfher…    0.000095
0xdf612c     0.000095
0xffe678     0.000095
0x118c16     0.000095
0x857d73     0.000095
Name: From, Length: 2056, dtype: float64

In [8]:
crypto_punks_df.To.value_counts(normalize=True)

0xc352b5     0.163967
WrappedC…    0.038292
0xb88f61     0.028964
0xcbd482     0.025200
0x577ebc     0.024873
               ...   
0xc50bc8     0.000164
0xe53ef2     0.000164
0xc04f55     0.000164
artofthe…    0.000164
0x857d73     0.000164
Name: To, Length: 1697, dtype: float64

In [9]:
crypto_punks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20002 entries, 0 to 20001
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  20002 non-null  int64 
 1   Type        20002 non-null  object
 2   From        10555 non-null  object
 3   To          6111 non-null   object
 4   Amount      14622 non-null  object
 5   Txn         20002 non-null  object
 6   Punk        20002 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 1.1+ MB


In [10]:
crypto_punks_df.Punk = crypto_punks_df.Punk.astype("category")

In [11]:
crypto_punks_df = crypto_punks_df[crypto_punks_df[['From', 'To']].notnull().any(1)]

In [12]:
ether_amount = []
import re
for i in crypto_punks_df['Amount'].values.tolist():
    if isinstance(i, str):
        i = i[0:i.index("(")-2]
        if bool(re.search(r'[A-Z]',i)):
                i = i.replace("K", "000")
                i = i.replace(",","")
        else:
            i = i.replace(",", ".")
        
        i = i.replace("<","")
        if 'Y' in i:
            i = i.replace("Y","")
        ether_amount.append(i)
    else:
        ether_amount.append(i)

In [13]:
crypto_punks_df['ether_amount'] = ether_amount

In [14]:
crypto_punks_df[crypto_punks_df['ether_amount']=='10Y']

,Unnamed: 0,Type,From,To,Amount,Txn,Punk,ether_amount


In [15]:
crypto_punks_df

,Unnamed: 0,Type,From,To,Amount,Txn,Punk,ether_amount
0,0,Claimed,NaN,0xc352b5,NaN,"Jun 23, 2017",0,NaN
1,1,Bid,0x948a10,NaN,0.67Ξ ($219),"Jun 23, 2017",0,0.67
2,2,Bid,0x00d7c9,NaN,0.98Ξ ($320),"Jun 23, 2017",0,0.98
3,3,Sold,0xc352b5,0x00d7c9,0.98Ξ ($320),"Jun 23, 2017",0,0.98
4,4,Bid,0x717403,NaN,0.05Ξ ($14),"Jun 25, 2017",0,0.05
...,...,...,...,...,...,...,...,...
19990,4,Sold,0x00b278,0xec46dd,"67.80Ξ ($225,755)","Aug 23, 2021",1633,67.80
19994,8,Bid,0x2891a4,NaN,"90Ξ ($340,024)","Sep 02, 2021",1633,90
19995,9,Bid Withdrawn,0x2891a4,NaN,"90Ξ ($344,656)","Sep 02, 2021",1633,90
19999,13,Sold,0xec46dd,0x04fd71,"100Ξ ($385,975)","Sep 04, 2021",1633,100


In [16]:
crypto_punks_df['ether_amount'].str.isalnum()

0          NaN
1        False
2        False
3        False
4        False
         ...  
19990    False
19994     True
19995     True
19999     True
20001      NaN
Name: ether_amount, Length: 12267, dtype: object

In [17]:
crypto_punks_df['ether_amount'] = crypto_punks_df['ether_amount'].astype(float)

In [18]:
def get_modes(df, feature_list,number_of_modes=2):
    mode_1_values = []
    mode_1_freq = []
    mode_2_values = []
    mode_2_freq = []
    for feature in feature_list:
        modes = df[feature].value_counts()[0:number_of_modes]
        if len(modes) > 1:
            mode_1_values.append(modes.index.values.tolist()[0])
            mode_1_freq.append(modes.values.tolist()[0])
            mode_2_values.append(modes.index.values.tolist()[1])
            mode_2_freq.append(modes.values.tolist()[1])
        else:
            mode_1_values.append(modes.index.values.tolist()[0])
            mode_1_freq.append(modes.values.tolist()[0])
            mode_2_values.append(None)
            mode_2_freq.append(None)
    return mode_1_values, mode_1_freq, mode_2_values, mode_2_freq 

In [19]:
mode_1_values, mode_1_freq, mode_2_values, mode_2_freq = get_modes(crypto_punks_df, feature_list=["Punk","Type","To","From", "Txn", "Amount"])
categorical_dqr = crypto_punks_df[["Punk","Type","To","From", "Txn", "Amount"]].describe().T
categorical_dqr['missing_count'] = crypto_punks_df.isna().sum()
categorical_dqr['%_missing'] = categorical_dqr['missing_count'] / crypto_punks_df.count()[1]
categorical_dqr['Mode'] = mode_1_values
categorical_dqr['Mode Freq'] = mode_1_freq
categorical_dqr['Mode %'] = categorical_dqr['Mode Freq'] / categorical_dqr['count']
categorical_dqr['2nd Mode'] = mode_2_values
categorical_dqr['2nd Mode Freq'] = mode_2_freq
categorical_dqr['2nd Mode %']= categorical_dqr['2nd Mode Freq'] / categorical_dqr['count']
categorical_dqr[['count', '%_missing', 'unique',"Mode","Mode Freq","Mode %","2nd Mode","2nd Mode Freq","2nd Mode %"]].rename(columns={'unique': "Card.", "count":"Count"})

,Count,%_missing,Card.,Mode,Mode Freq,Mode %,2nd Mode,2nd Mode Freq,2nd Mode %
Punk,12267,0.000000,1635,635,79,0.00644004,339,50,0.00407598
Type,12267,0.000000,9,Bid,3812,0.310752,Bid Withdrawn,2340,0.190756
To,6111,0.501834,1697,0xc352b5,1002,0.163967,WrappedC…,234,0.0382916
From,10555,0.139561,2056,0xc352b5,634,0.0600663,Pranksy,365,0.0345808
Txn,12267,0.000000,875,"Jun 23, 2017",1640,0.133692,"Feb 04, 2021",275,0.0224179
Amount,7912,0.355018,5964,0.05Ξ ($14),131,0.0165571,0.05Ξ ($13),95,0.0120071


In [20]:
numeric_dqr = crypto_punks_df.describe(include=np.number).T
numeric_dqr['missing_count'] = crypto_punks_df.isna().sum()
numeric_dqr['%_missing'] = numeric_dqr['missing_count'] / crypto_punks_df.count()[0]
numeric_dqr['Card.'] = crypto_punks_df[numeric_dqr.T.columns.values.tolist()].nunique()
numeric_dqr[['count', '%_missing', 'Card.', 'min', '25%', 'mean', '50%', '75%', 'max', 'std']].rename(columns={'25%': '1st Qtr.', '50%': 'Median', '75%': '3rd Qtr.'})

,count,%_missing,Card.,min,1st Qtr.,mean,Median,3rd Qtr.,max,std
Unnamed: 0,12267.0,0.000000,108,0.0,2.00,1.103685e+01,6.00,15.0,1.310000e+02,1.435400e+01
ether_amount,7912.0,0.355018,934,0.0,0.57,5.848862e+31,13.25,37.0,1.157903e+35,2.600772e+33


In [21]:
crypto_punks_df.to_csv('/data/cleaned_punks.csv')

From the Initial EDA, we can see that there are missing values in both the To and From columns. We've excluded all transactions from the data where both the To & From values are missing, as these transactions are seen as non successful. We can see that 50% of the transactions have missing values for the To column when all unsuccessful transactions were excluded. This could be due to the most common type of transaction being Bid as shown by the 1st Mode of the Type feature, accounting for 31% of all transactions. We also see that the most transactions occurred on June 23,2017. The Amount feature contains information about not just he crypto amount, but also the dollar value. Some data manipulation has applied to engineer the feature Ether_amount. This shows us the descriptive statistics for the numeric value  of Ether_amount. We can see there are 35% of all transactions with no Ether_amount, indicating that these were claims on a NFT. When looking at the distribution of Ether_amount we can see that 75% of all transactions are equal to or less than 37.0 Ether. With the maximum being order of magnitude larger, indicating a skew in the data, with an outlier in the data set. This however isn't a concern as we are not using this feature to model or predict any outcome. 

In [2]:
from py2neo import Graph

In [3]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "test"))

## Question 3 - Data Import

In [ ]:
graph.run('LOAD CSV WITH HEADERS FROM "file:///data/cleaned_punks.csv" AS row \
WITH row WHERE row.From IS NOT NULL \
MERGE (punk:Punk {id:row.Punk}) \
MERGE (wallet: Wallet {id:row.From}) \
MERGE (wallet)-[rel:TRANSACTS {type:row.Type,date:row.Txn, amount: coalesce(row.Amount,"none")}]->(punk);
')

In [ ]:
graph.run('LOAD CSV WITH HEADERS FROM "file:///data/cleaned_punks.csv" AS row \
WITH row WHERE row.To IS NOT NULL \
MERGE (wallet: Wallet{id: row.To}) \
MERGE (punk: Punk{id: row.Punk}) \
MERGE (punk)-[rel:TRANSACTS {type: row.Type, date:row.Txn, amount: coalesce(row.Amount,"none")}]->(wallet);')

## Question 4 - Which wallets hold the largest number of assets/Cryptopunks?

In [5]:
result = graph.run('MATCH (p:Punk)-[txn_:TRANSACTS]->(to:Wallet) \
RETURN  to.id, txn_.type, count(*) AS count_ \
ORDER BY count_ DESC \
LIMIT 10;')

records = list(result)
print(len(records))
for record in records:
    print(record)

10
'0xc352b5'	'Claimed'	1000
'WrappedC…'	'(Wrap)'	225
'0xb88f61'	'Transfer'	177
'0xcbd482'	'Claimed'	154
'0x35388c'	'Transfer'	151
'0x2cbee4'	'Transfer'	150
'0x577ebc'	'Transfer'	146
'Pranksy'	'Transfer'	57
'0x938bd2'	'Transfer'	53
'0x4d8e16'	'Transfer'	47


## Question 5 - Cycles

In [6]:
graph.run('MATCH p=(n)-[*1..2]->(n) RETURN count(p);')

count(p)
11628


## Question 6 - most traded

In [7]:
graph.run('MATCH (Wallet)-[:TRANSACTS]-(Punk)-[:TRANSACTS]->(:Wallet) \
RETURN Punk, COLLECT(Wallet) as wallets_that_held \
ORDER BY SIZE(wallets_that_held) DESC LIMIT 1;')

Punk                     | wallets_that_held                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

## Question 7- Page Rank

In [12]:
graph.run("CALL gds.graph.create('PageRank', ['Wallet', 'Punk'],'TRANSACTS');")

nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,createMillis
"{Punk: {label: 'Punk', properties: {}}, Wallet: {label: 'Wallet', properties: {}}}","{TRANSACTS: {orientation: 'NATURAL', aggregation: 'DEFAULT', type: 'TRANSACTS', properties: {}}}",PageRank,4111,16585,129


In [13]:
result = graph.run("CALL gds.pageRank.stream('PageRank') YIELD nodeId, score RETURN gds.util.asNode(nodeId).id AS name, score ORDER BY score DESC, name ASC;")
records = list(result)
print(len(records))
for record in records:
    print(record)

4111
'0xc352b5'	220.72927744067727
'WrappedC…'	44.89781386980504
'0xb88f61'	42.88111592736094
'0x35388c'	27.753684071113156
'0x2cbee4'	27.599596433820277
'0xcbd482'	23.111350156757688
'0xe34bde'	22.956622989740854
'897'	22.671764073412344
'0x577ebc'	22.611485880370843
'938'	20.211060771929944
'Pranksy'	13.179632836472226
'0xfd9065'	11.409743678768708
'0xfeeb2d'	10.124858493202877
'0x4d8e16'	9.454530214169525
'0xcc7c33'	8.425285377312985
'Hemba'	7.50820997132703
'639'	7.403912864637842
'0x48db19'	7.242698294330361
'0x269616'	7.200718910066767
'441'	6.612002671350071
'483'	6.510061833581511
'0x938bd2'	6.469898008986706
'1369'	6.013518702633908
'0x00bd9f'	5.534843192689674
'1417'	5.316828372616097
'0xa3cb30'	5.30717067891591
'punksotc…'	5.278704676506187
'0x6611fe'	5.101570854202039
'743'	4.827840436934858
'824'	4.416951619483458
'0xe4f0ac'	4.335357182898492
'1436'	4.165290783769832
'alien635…'	4.156641947218152
'ddaavvee'	4.059667473365982
'1'	3.9794040855159505
'1376'	3.893529323612098


'1568'	0.3162793671993638
'1596'	0.3162793671993638
'172'	0.3157004914120499
'0x8814c8'	0.3152821665202251
'0x86292f'	0.3152323426916535
'0xf18fce'	0.3152323426916535
'0xccc003'	0.3150638304214862
'0x73d64b'	0.3147728008860187
'fth.eth'	0.3147728008860187
'0x525ba4'	0.3145038639503009
'0xf09d1a'	0.3145038639503009
'0xf224d5'	0.3145038639503009
'dan_'	0.3145038639503009
'jomessin'	0.3145038639503009
'moonphas…'	0.3145038639503009
'naty.eth'	0.3145038639503009
'0x1b3321'	0.3142163086723105
'0x6ff0be'	0.3142163086723105
'0xa22724'	0.3142163086723105
'0xcbbc7b'	0.3142163086723105
'0xce18b6'	0.3142163086723105
'0xee707d'	0.3142163086723105
'owl.eth'	0.3142163086723105
'0x88d357'	0.31420473637655116
'0xf8be95'	0.31420473637655116
'0x304288'	0.31371889845877143
'PM_disco…'	0.31371889845877143
'1012'	0.3136628615831208
'0x05083f'	0.31352573894248525
'0x26dee6'	0.3133753926032556
'foobarde…'	0.3133753926032556
'marcor.e…'	0.3133753926032556
'AlottaMo…'	0.3132371648259735
'0x579247'	0.3132158151

The PageRank algorithm is mainly concerned with the centrality of a given node. Analyzing the results from the PageRank algorithm, we can see that the '0xc352b5' Wallet has the highest score given by the PageRank algorithm. This is due to the fact that the wallet is connected to 77 unique Punks, that are then in turn connected to other Wallets. Due to the specific data modelling used, relationships or edges exist showing e.g. a wallet claimed a punk, and a punk was claimed by a wallet, addind more incoming relationships to a single node. This however isn't too much of a worry as it is a property of the data modelling present for all nodes. 

The PageRank algorithm results can be interpretted as the importance of a node measured by the number of incoming relationships of a single node. As mentioned the highest scoring wallet, '0xc352b5', has the most incoming nodes, which in turns trades the NFT's to other nodes. This result is also supported by the finding of the Exploratory Data Analysis on the input data, where this specified wallet accounts for 16.39% transactions where the wallet is receiving the Punk, and accounts for 60% of transactions where the wallet is trades away the NFT. 